### Notebook utilizado para criar a base de dados da RAIS com as informações que serão utilizadas.

#### Os arquivos da RAIS foram baixados via ftp pelo link: ftp://ftp.mtps.gov.br/pdet/microdados/
#### Em seguida, foram extraídos na pasta local E:/Apps/Dados/Rais
##### Os arquivos não foram incluídos no projeto por serem muito grandes.

In [1]:
# Importando bibliotecas
import pandas as pd
import pickle

In [2]:
# Definindo as colunas que serão utilizadas.
columns = ['Município', 'CNAE 2.0 Classe', 'Vínculo Ativo 31/12', 'Vl Remun Dezembro Nom']
# Lista de sufixos dos arquivos.
arqs = ['CENTRO_OESTE','MG_ES_RJ','NI','NORDESTE','NORTE','SP','SUL']
# Ano da Rais que será tratado.
ano = '2022'

In [3]:
# Criando a base de dados da RAIS

# Cria o DataFrame que será utilizado no final.
df_final = pd.DataFrame()
# Cria uma lista para receber os locais dos arquivos temporários.
list_files = []

for a in arqs:
    print('Iniciando', a)
    path = 'E:/Apps/Dados/Rais/{0}/RAIS_VINC_PUB_{1}.txt'.format(ano,a)
    
    # Como os arquivos são muito grandes eles serão tratados em partes, sendo divididos a cada 2 milhões de linhas.
    
    print('Tratando partes do arquivo:')
    list_chunks = []
    for i, chunk in enumerate(pd.read_csv(path, delimiter=';', encoding='latin-1', usecols=columns, low_memory=False, chunksize=2000000)):
        # Tornando o valor de 'Vl Remun Dezembro Nom' em um float.
        chunk['Vl Remun Dezembro Nom'] = chunk['Vl Remun Dezembro Nom'].apply(lambda x: x.replace(',','.')).astype('float')
        # Adicionando um contator para os vínculos ativos em 31/12.
        rais_at = chunk[chunk['Vínculo Ativo 31/12'] == 1]
        # Agrupando por atividade econômica e município.
        chunk_res = rais_at.groupby(by=['CNAE 2.0 Classe', 'Município'], as_index=False).sum()
        # Renomeando as colunas.
        chunk_res.rename(columns = {'CNAE 2.0 Classe': 'ID CNAE', 'Vínculo Ativo 31/12': 'Número Empregos', 'Vl Remun Dezembro Nom': 'Massa Salarial'}, inplace=True)
        list_chunks.append(chunk_res)
        print('{0} - Concluída.'.format(i))
    # Unifica as partes tratadas do arquivo em um único DataFrame e salva um arquivo temporário em pickle.
    df = pd.concat(list_chunks)
    temp_file = 'E:/Apps/Dados/Rais/{0}/Tratados/tratado{0}{1}.pkl'.format(ano,a)
    df.to_pickle(temp_file)
    list_files.append(temp_file)
    print('Finalizado', a)
    print('__________________')

# Para finalizar é feita a concatenação no df_final e ele é salvo.
df_final = pd.concat([pd.read_pickle(f) for f in list_files])
df_final.to_pickle('../assets/rais{0}.pkl'.format(ano),  compression='gzip')

Iniciando  CENTRO_OESTE
Tratando partes do arquivo:
0 - Concluída.
1 - Concluída.
2 - Concluída.
3 - Concluída.
Finalizado  CENTRO_OESTE
__________________
Iniciando  MG_ES_RJ
Tratando partes do arquivo:
0 - Concluída.
1 - Concluída.
2 - Concluída.
3 - Concluída.
4 - Concluída.
5 - Concluída.
6 - Concluída.
7 - Concluída.
8 - Concluída.
Finalizado  MG_ES_RJ
__________________
Iniciando  NI
Tratando partes do arquivo:
0 - Concluída.
Finalizado  NI
__________________
Iniciando  NORDESTE
Tratando partes do arquivo:
0 - Concluída.
1 - Concluída.
2 - Concluída.
3 - Concluída.
4 - Concluída.
5 - Concluída.
6 - Concluída.
Finalizado  NORDESTE
__________________
Iniciando  NORTE
Tratando partes do arquivo:
0 - Concluída.
1 - Concluída.
2 - Concluída.
Finalizado  NORTE
__________________
Iniciando  SP
Tratando partes do arquivo:
0 - Concluída.
1 - Concluída.
2 - Concluída.
3 - Concluída.
4 - Concluída.
5 - Concluída.
6 - Concluída.
7 - Concluída.
8 - Concluída.
9 - Concluída.
10 - Concluída.
11